In [1]:
import random,math,copy,timeit,operator
from sklearn import utils,preprocessing
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from copy import deepcopy as dc
from sys import exit
from sklearn.neighbors import KNeighborsClassifier
from sklearn import model_selection as ms

In [2]:
class ants(object):
    def __init__(self,gens=None,mse=None):
        self.gens=gens
        self.mse=mse

In [3]:
def random_search(n,dim):
    gens=[[0 for g in range(dim)] for _ in range(n)]
    for i,gen in enumerate(gens) :
        r=random.randint(1,dim)
        for _r in range(r):
            gen[_r]=1
        random.shuffle(gen)
    return gens



In [4]:
def k_search(n,dim,flist,pheromone):
    gens=[[0 for g in range(dim)] for _ in range(n)]
    for i,gen in enumerate(gens) :
        r=random.randint(1,dim)
        for _r in range(r):
            gen[_r]=1
        random.shuffle(gen)
        for j,x in enumerate(gen):
            gen[j]=flist[j]*gen[j]
    return gens


In [5]:
def evaluate(train_d,train_l,gen):
        mask=np.array(gen) > 0
        al_data=np.array([al[mask] for al in train_d])
        kf = ms.KFold(n_splits=4)
        s = 0
        for tr_ix,te_ix in kf.split(al_data):
            s+= KNeighborsClassifier(n_neighbors=3).fit(al_data[tr_ix],train_l[tr_ix]).score(al_data[te_ix],train_l[te_ix])#.predict(al_test_data)
        s/=4
        return s

In [6]:
def BACO(train_d,train_l,n=30,max_iter=25,T0=0.1,k=10,m=25,p=3,rho=0.25):
    """
    input:{ Eval_Func: Evaluate_Function, type is class
            n: Number of population, default=20
            max_iter: Number of max iteration, default=300
            }
    output:{
            Best position: type list(int) [1,0,0,1,.....]
            Nunber of 1s in best position: type int [0,1,1,0,1] → 3
            }
    """
    dim=len(train_d[0])
    global_best=float("-inf")
    pheromone=list([T0]*dim)
    pher_diff=list([0]*dim)
    global_position=tuple([0]*dim)
    antslist=[]
    gens_dict = {}
    for i in range(n):
        gens=random_search(m,dim)
        mse=0
        for gen in gens:
            if tuple(gen) in gens_dict:
                score = gens_dict[tuple(gen)]
            else:
                score=evaluate(train_d,train_l,gen)
                gens_dict[tuple(gen)]=score
            global_best=score
            global_position=dc(gen)
            mse+=score
        mse=mse/m
        antslist.append(ants(gens,mse))
    antslist.sort(key=operator.attrgetter('mse'), reverse=True)
    klist=list(antslist[i] for i in range(k))
    mselist=list(antslist[i].mse for i in range(k))
    for j in range(len(klist)):
        diff=(max(mselist)-mselist[j])/(max(max(mselist)-mselist))
        flist=list([0]*dim)
        for gen in klist[j].gens:
            for a,x in enumerate(gen):
                if x==1:
                    pher_diff[a]=diff
                    flist[a]=1
                else:
                    pher_diff[a]=0
    
    for i in range(len(pheromone)):
        pheromone[i]=pheromone[i]*rho + pher_diff[i]
        
        
    for it in range(max_iter):
        m=m-p
        for i in range(n):
            gens=k_search(m,dim,flist,pheromone)
            mse=0
            for gen in gens:
                if tuple(gen) in gens_dict:
                    score = gens_dict[tuple(gen)]
                else:
                    score=evaluate(train_d,train_l,gen)
                    gens_dict[tuple(gen)]=score
                global_best=score
                global_position=dc(gen)
                mse+=score
            mse=mse/m
            antslist.append(ants(gens,mse))
        antslist.sort(key=operator.attrgetter('mse'), reverse=True)
        klist=list(antslist[i] for i in range(k))
        mselist=list(antslist[i].mse for i in range(k))
        for j in range(len(klist)):
            diff=(max(mselist)-mselist[j])/(max(max(mselist)-mselist))
            flist=list([0]*dim)
            for gen in klist[j].gens:
                for i,x in enumerate(gen):
                    if x==1:
                        pher_diff[i]=diff
                        flist[i]=1
                    else:
                        pher_diff[i]=0
    
        for i in range(len(pheromone)):
            pheromone[i]=pheromone[i]*rho + pher_diff[i]

        
    return global_position,global_position.count(1)


In [7]:
def test_score(gen,tr_x,tr_y,te_x,te_y):
    mask=np.array(gen) == 1
    al_data=np.array(tr_x[:,mask])
    al_test_data=np.array(te_x[:,mask])
    return np.mean([KNeighborsClassifier(n_neighbors=3).fit(al_data,tr_y).score(al_test_data,te_y) for i in range(5)])

In [8]:
with open("/home/shubbham28/Downloads/Breast Cancer/wdbc1.csv") as f:
    x=np.array([[float(d) for d  in data.split(',')] for data in f.read().splitlines()])
with open("/home/shubbham28/Downloads/Breast Cancer/wdbc2.csv") as f:
    y=np.array([float(data) for data in f.read().splitlines()])
lab_enc = preprocessing.LabelEncoder()
training_scores_encoded = lab_enc.fit_transform(y)
train_d, test_d, train_l, test_l = train_test_split(x, training_scores_encoded, test_size=0.25)

In [10]:
k=[1 for r in range(len(x[0]))]
print test_score(k,train_d,train_l,test_d,test_l)
fattr=0
ftest=0.0
flist=[0 for r in range(len(x[0]))]
final_list=[0 for r in range(len(x[0]))]
start=timeit.default_timer()
for i in range(20):
    g,l=BACO(train_d,train_l,n=30,max_iter=25,T0=0.1,k=10,m=25,p=3,rho=0.25)
    fattr+=l
    test=test_score(g,train_d,train_l,test_d,test_l)
    ftest+=test
    for j in range(len(flist)):
        if g[j]==1:
            flist[j]+=1
    print("{0}  {1}  {2}  {3:.6f}".format(i+1,"".join(map(str,g)),l,test))
fattr=fattr/20
ftest=ftest/20
end=timeit.default_timer()
time=end-start
final=np.argsort(flist)[::-1][:fattr]
for i in range(len(final)):
    final_list[final[i]]=1
print("{0}  {1}   {2}   {3:.6f}    {4:.4f}".format("Final: ","".join(map(str,final_list)),fattr,ftest,time))

0.9300699300699302
1  111111111111111111111111111101  29  0.930070
2  000101000111101111111111100010  18  0.930070
3  111111011110100111111010101011  22  0.916084
4  110001101011000010000110000011  12  0.923077
5  111101111111111101110111111011  26  0.930070
6  001010010100111111010000000101  13  0.818182
7  111111111111111111111111111111  30  0.930070
8  111001111011010001000111111101  19  0.930070
9  111100111111111101011010110110  22  0.916084
10  111111011110111111111111111011  27  0.930070
11  101011100011011011101001010100  16  0.916084
12  011001110101001110101101011101  18  0.923077
13  111111101110101111110111111111  26  0.930070
14  000100000001101000000000000000  4  0.881119
15  001000100000100100100001010000  7  0.916084
16  110111111011111011011111111111  26  0.930070
17  001010100011000001001010110101  12  0.902098
18  011011011010011111101111110100  20  0.930070
19  001100100000011000000110010101  10  0.937063
20  000000100000000000100000000000  2  0.867133
Final:   0110

In [15]:
with open("/home/shubbham28/Downloads/Arrhythmia/heart.csv") as f:
    x=np.array([[float(d) for d  in data.split(',')] for data in f.read().splitlines()])
with open("/home/shubbham28/Downloads/Arrhythmia/heart_output.csv") as f:
    y=np.array([float(data) for data in f.read().splitlines()])
lab_enc = preprocessing.LabelEncoder()
training_scores_encoded = lab_enc.fit_transform(y)
train_d, test_d, train_l, test_l = train_test_split(x, training_scores_encoded, test_size=0.25)
y.shape

(270,)

In [14]:
k=[1 for r in range(len(x[0]))]
print test_score(k,train_d,train_l,test_d,test_l)
fattr=0
ftest=0.0
flist=[0 for r in range(len(x[0]))]
final_list=[0 for r in range(len(x[0]))]
start=timeit.default_timer()
for i in range(5):
    g,l=BACO(train_d,train_l,n=30,max_iter=25,T0=0.1,k=10,m=25,p=3,rho=0.25)
    fattr+=l
    test=test_score(g,train_d,train_l,test_d,test_l)
    ftest+=test
    for j in range(len(flist)):
        if g[j]==1:
            flist[j]+=1
    print("{0}  {1}  {2}  {3:.6f}".format(i+1,"".join(map(str,g)),l,test))
fattr=fattr/5
ftest=ftest/5
end=timeit.default_timer()
time=end-start
print flist,fattr
final=np.argsort(flist)[::-1][:fattr]
print final
for i in range(len(final)):
    final_list[final[i]]=1
print("{0}  {1}   {2}   {3:.6f}    {4:.4f}".format("Final: ","".join(map(str,final_list)),fattr,ftest,time))

0.6911764705882353
1  1111110111111  12  0.691176
2  0000001001000  2  0.691176
3  0100111111110  9  0.602941
4  1111001011110  9  0.617647
5  1111111111001  11  0.691176
[3, 4, 3, 3, 3, 3, 4, 3, 4, 5, 3, 3, 2] 8
[ 9  8  6  1 11 10  7  5]
Final:   0100011111110   8   0.658824    43.6208
